In [6]:
import pandas as pd
from sklearn.neural_network import MLPRegressor

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Minas.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Fecha'] = pd.to_datetime(indicadores_df['Fecha'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])

# Definir subprocesos e indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha

# Calcular diferentes tipos de correlación
correlation_methods = {
    'Pearson': df_unificado.corr(method='pearson'),
    'Spearman': df_unificado.corr(method='spearman'),
    'Kendall': df_unificado.corr(method='kendall')
}

# Resultados para cada método
resultados_comparacion = {}

for method, correlation_matrix in correlation_methods.items():
    correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
    top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

    # Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
    train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
    test_data = df_unificado[(df_unificado['Subproceso_Costo'] >= '2022-01-01') & (df_unificado['Subproceso_Costo'] < '2023-01-01')]

    # Modelo Neural Network (Redes Neuronales)
    resultados_nn = {}
    nn_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
    
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            y_test = test_data[subproceso]
            nn_model.fit(X_train, y_train)
            y_pred_2022 = nn_model.predict(X_test)
            
            # Comparación mes a mes
            resultados_nn[subproceso] = {
                'Predicción 2022': y_pred_2022, 
                'Costo Real 2022': y_test.values, 
                'Diferencia': y_pred_2022 - y_test.values, 
                'Diferencia %': ((y_pred_2022 - y_test.values) / y_test.values) * 100
            }
    
    # Guardar los resultados para este método de correlación
    resultados_comparacion[method] = pd.DataFrame(resultados_nn)

# Selección del mejor modelo para cada subproceso basado en las predicciones de todo 2022
mejor_modelo_por_subproceso = pd.DataFrame(columns=['Subproceso', 'Mejor Modelo', 'Diferencia Media %'])

for subproceso in subprocesos_cols:
    mejor_diferencia_pct_media = float('inf')
    mejor_metodo = None
    
    for method, resultados_df in resultados_comparacion.items():
        if subproceso in resultados_df.index:
            diferencia_pct_media = resultados_df.loc[subproceso, 'Diferencia %'].abs().mean()
            if diferencia_pct_media < mejor_diferencia_pct_media:
                mejor_diferencia_pct_media = diferencia_pct_media
                mejor_metodo = method
    
    mejor_modelo_por_subproceso = pd.concat([
        mejor_modelo_por_subproceso,
        pd.DataFrame({
            'Subproceso': [subproceso],
            'Mejor Modelo': [mejor_metodo],
            'Diferencia Media %': [mejor_diferencia_pct_media]
        })
    ], ignore_index=True)

# Mostrar los resultados
print("Mejor modelo para cada subproceso en 2022:")
print(mejor_modelo_por_subproceso)

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1092\2130086436.py:57: RuntimeWarning: divide by zero encountered in divide
  'Diferencia %': ((y_pred_2022 - y_test.values) / y_test.values) * 100
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  wa

Mejor modelo para cada subproceso en 2022:
                         Subproceso Mejor Modelo  Diferencia Media %
0       ADM OPERACIÓN Y GESTIÓN ESP         None                 inf
1   ADM OPERACIÓN Y GESTIÓN ESP SUR         None                 inf
2     ADM OPERACIÓN Y GESTIÓN LLANO         None                 inf
3       ADM OPERACIÓN Y GESTIÓN MIR         None                 inf
4       ADM OPERACIÓN Y GESTIÓN OXE         None                 inf
..                              ...          ...                 ...
56                  TRONADURA LLANO         None                 inf
57                TRONADURA MIRADOR         None                 inf
58         TRONADURA TESORO CENTRAL         None                 inf
59        TRONADURA TESORO NOR ESTE         None                 inf
60             TRONADURA TESORO SUR         None                 inf

[61 rows x 3 columns]


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1092\2130086436.py:57: RuntimeWarning: divide by zero encountered in divide
  'Diferencia %': ((y_pred_2022 - y_test.values) / y_test.values) * 100
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1092\2130086436.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mejor_modelo_por_subproceso = pd.concat([
